In [17]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import matplotlib.pylab as plt
import scipy.sparse as sparse
import numpy as np

In [18]:
from dotenv import dotenv_values

config = dotenv_values("../.env")

In [142]:
book_df = pd.read_parquet('../data/book_eng.parquet')
book_df

,Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,...,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,PagesNumber,Description,pagesNumber,Count of text reviews
13,1900574,The Political and Social Ideas of Jules Verne,Jean Chesneaux,0500010846,0.00,1972,10,2,Thames and Hudson,5:0,...,3:0,2:0,1:0,total:0,0,eng,224.0,Translated from Une lecture politique de Jules...,NaN,NaN
18,1900581,Risky Business,Nora Roberts,0786253843,3.87,2003,8,1,Thorndike Press,5:1826,...,3:1483,2:392,1:89,total:5503,3,eng,344.0,Liz Palmer is raising her daughter alone becau...,NaN,NaN
20,1900585,"His Brain, Her Brain: How Divinely Designed Di...",Walt Larimore,031024028X,4.19,2008,1,27,Zondervan,5:115,...,3:30,2:8,1:8,total:240,30,en-US,233.0,"She reads people, and he reads manualsHe doesn...",NaN,NaN
25,1900601,The Secrets of Rosslyn,Roddy Martine,1841585904,3.78,2008,1,1,Birlinn Publishers,5:6,...,3:11,2:1,1:0,total:32,5,eng,224.0,For generations the tiny chapel of Rosslyn has...,NaN,NaN
40,1900644,Oranges from Spain,David Park,0747571627,3.31,2004,4,1,Bloomsbury Publishing PLC,5:8,...,3:11,2:6,1:4,total:39,6,eng,192.0,A collection of stories about of the trials of...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850066,1499848,Makeover Magic,Kelly McKain,0746066899,4.07,2005,26,8,Usborne,5:89,...,3:29,2:16,1:5,total:182,12,en-GB,NaN,"A fun and funky new series charting the hopes,...",160.0,12.0
1850067,1499849,The Goddess Society,Kelly McKain,0439963699,3.24,2005,21,1,Scholastic Point,5:8,...,3:24,2:11,1:4,total:66,1,eng,NaN,None,288.0,1.0
1850070,1499854,The Moonstone Mass and Others,Harriet Prescott Spofford,1553100085,3.50,2000,15,9,Ash-Tree Press,5:1,...,3:1,2:1,1:0,total:4,0,eng,NaN,"In 1859, a twenty-four year old author burst u...",183.0,0.0
1850080,1499880,Boy,James Hanley,1847490069,3.61,2007,1,4,Oneworld Classics,5:31,...,3:54,2:11,1:5,total:152,17,eng,NaN,Acclaimed by luminaries such as William Faulkn...,220.0,17.0


In [151]:
book_df_cleaned = book_df.dropna(subset = ['Description'])
book_df_cleaned.reset_index(drop = True, inplace = True)

In [152]:
book_df.sort_values('RatingDistTotal', ascending = False)[:1000]

,Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,...,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,PagesNumber,Description,pagesNumber,Count of text reviews
593218,2339477,The Last Report on the Miracles at Little No H...,Louise Erdrich,0061577626,4.19,2016,8,16,Harper Perennial,5:4531,...,3:1463,2:410,1:131,total:9999,50,eng,361.0,<strong>A <em>New York Times</em> Notable Book...,NaN,NaN
1086877,310258,"The Snowy Day (Peter, #1)",Ezra Jack Keats,0140501827,4.16,1976,28,10,Puffin Books,5:49230,...,3:17356,2:4842,1:2193,total:99986,3220,eng,NaN,None,40.0,NaN
1775322,30189,"Cast in Courtlight (Chronicles of Elantra, #2)",Michelle Sagara,0373802447,4.11,2006,18,7,Luna,5:3737,...,3:1810,2:298,1:71,total:9998,296,eng,NaN,None,488.0,NaN
1053279,549129,"Therapy (Alex Delaware, #18)",Jonathan Kellerman,0739313304,3.82,2004,20,4,Random House Audio,5:2346,...,3:3069,2:424,1:78,total:9997,4,eng,NaN,None,0.0,NaN
940443,1829655,The Demon's Lexicon,Sarah Rees Brennan,1416963790,3.73,2009,6,2,Margaret K. McElderry Books,5:2770,...,3:2512,2:846,1:438,total:9997,1011,eng,336.0,"Sixteen-year-old Nick and his brother, Alan, a...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447159,898167,A History of the Ancient World: Volume II Rome,Michael Rostovtzeff,0195002245,3.95,1960,31,12,"Oxford University Press, USA",5:29,...,3:24,2:5,1:0,total:98,2,eng,NaN,First published in 1927 this monumental book h...,368.0,2.0
1552620,4225565,The Talleyrand Maxim,J.S. Fletcher,1406941115,3.72,2006,11,3,Hard Press,5:20,...,3:31,2:5,1:2,total:98,1,eng,180.0,"Nesta, left alone, gave herself up to deep tho...",NaN,NaN
1501041,4066740,Introductory Quantum Mechanics,Richard L. Liboff,0201878798,3.66,1997,6,23,Addison Wesley Publishing Company,5:21,...,3:26,2:11,1:2,total:98,0,eng,874.0,None,NaN,NaN
1059183,559401,Blood + Water,Judd Winick,140120175X,3.43,2009,20,10,Vertigo,5:11,...,3:43,2:9,1:2,total:98,7,en-US,NaN,None,128.0,NaN


In [22]:
sample_book_cleaned = book_df_cleaned.sample(100, random_state=42)

### Generate Description Embeddings

#### TF-IDF

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), stop_words='english')
tf_matrix = tf.fit_transform(sample_books_cleaned['Description'])
tf.get_feature_names_out()
tf_matrix.shape

(63, 12399)

In [55]:
pd.DataFrame(tf_matrix.todense(), columns=tf.get_feature_names(), index=sample_books_cleaned.Name)

/Users/seivabeljessica/opt/miniconda3/envs/ds/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,000,000 entry,000 entry index,000 years,000 years artistic,100,100 questions,100 questions asked,100 success,100 success secrets,...,young interference experiment,young old,young old mother,young old proclaim,épico,épico que,épico que relata,último,último año,último año la
Name,,,,,,,,,,,,,,,,,,,,,
1632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Iliad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07409,0.07409,0.07409,0.07409,0.07409,0.07409
A Children's Book of Verse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
"Systems Analysis and Modeling in Defense: Developments, Trends, and Issues",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
"Poverty, Inequality and Health: An International Perspective",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Climates of the British Isles: Present, Past and Future",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Paperwork,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
The Corinne T. Netzer Carbohydrate Gram Counter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


#### OpenAI Embeddings

In [11]:
import openai

In [12]:
# %%time

import multiprocessing
import pandas as pd
import numpy as np
from multiprocessing import Pool
import scipy.sparse as sp
import tiktoken

In [25]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [26]:
tokens = []
for desc in sample_books_cleaned['Description']:
    tokens.append(num_tokens_from_string(desc, "cl100k_base"))

tokens


[83,
 152,
 300,
 55,
 118,
 144,
 42,
 94,
 55,
 430,
 100,
 169,
 323,
 238,
 106,
 307,
 217,
 141,
 35,
 166,
 176,
 109,
 216,
 154,
 307,
 30,
 206,
 43,
 23,
 5,
 259,
 60,
 63,
 178,
 59,
 103,
 342,
 210,
 11,
 146,
 321,
 211,
 14,
 346,
 190,
 201,
 169,
 156,
 119,
 77,
 113,
 148,
 100,
 897,
 100,
 66,
 212,
 164,
 42,
 182,
 77,
 84,
 112,
 265,
 177,
 46,
 184,
 194,
 74,
 201,
 258,
 64,
 186,
 55,
 86,
 82,
 291,
 74,
 233,
 161,
 35,
 323,
 67,
 101,
 40,
 41,
 66,
 115,
 186,
 231,
 164,
 63,
 47,
 95,
 134,
 184,
 107,
 389,
 71,
 56]

In [29]:
print(np.mean(tokens))
print(np.max(tokens))

152.22
897


In [37]:
sample_books_cleaned.iloc[:20]['Description'].values.tolist()

['Olafur Eliasson belongs to the generation of artists who spent the 90s exploring and expanding the boundaries between art, science, and nature, and their subjective and objective perception. Using fog, water, plants, and soil, he and landscape architect Gunther Vogt completely transformed the rigorously concrete and glass Kunsthaus Bregenz, pictured here in objective installation photographs and subjective texts by the artist.',
 'This book is a detailed, accessible and comprehensive reference manual reflecting current guidance &amp; citing recent peer-reviewed evidence. It is written by and for radiographers. Through text and diagrams the fundamental skills and techniques for acquisition of high quality diagnostic images are explained and demonstrated; high quality ultrasound images throughout underpin instruction on accurate image interpretation and diagnosis. Inclusion of unusual and rare appearances allow the reader to avoid common pitfalls and resolve diagnostic dilemmas.<br />S

In [136]:
%%time

def create_embeddings_in_batches(documents, batch_size, model):
    all_embeddings = []
    
    # Split the documents into batches of the specified size
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        response = openai.embeddings.create(
            input=batch,
            model=model
        )
        # Extract the embeddings from the response and add them to the list
        batch_embeddings = response.data[0].embedding
        all_embeddings.append(batch_embeddings)
    
    return all_embeddings

# Your list of 1000 documents
documents = sample_books_cleaned.iloc[:2]['Description'].values.tolist()  # Replace with your actual documents

# Call the function with your documents, desired batch size, and the model you want to use
embeddings = create_embeddings_in_batches(documents, 1, "text-embedding-ada-002")

CPU times: user 10.2 ms, sys: 16.8 ms, total: 27 ms
Wall time: 648 ms


In [137]:
item_feature_matrix_array.shape

(100, 1536)

In [138]:
len(embeddings[0])

1536

In [139]:
item_feature_matrix_array = np.array(embeddings)

# Print the item feature matrix
print("Item Feature Matrix:\n", item_feature_matrix_array)

Item Feature Matrix:
 [[ 4.86428570e-03 -5.69997029e-03 -7.52449362e-03 ... -1.60877667e-02
   7.65101146e-03 -2.69949529e-02]
 [-2.28366200e-02  1.43984174e-02  1.26246996e-02 ... -2.76491344e-02
   2.68992171e-05 -1.64329764e-02]]


#### Embeddings Langchain

In [54]:
data = sample_books_cleaned

In [59]:
from langchain.embeddings.openai import OpenAIEmbeddings


model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name
)

In [63]:
data

,Id,Name,Authors,ISBN,Rating,PublishYear,PublishMonth,PublishDay,Publisher,RatingDist5,...,RatingDist3,RatingDist2,RatingDist1,RatingDistTotal,CountsOfReview,Language,PagesNumber,Description,pagesNumber,Count of text reviews
161930,655862,Olafur Eliasson: The Mediated Motion,Eckhard Schneider,3883755052,4.67,2002,2,2,Walther Konig Verlag,5:2,...,3:0,2:0,1:0,total:3,0,None,NaN,Olafur Eliasson belongs to the generation of a...,73.0,NaN
1300390,3217053,"Breast Ultrasound: How, Why and When",Anne-Marie Dixon,0443100764,0.00,2007,12,1,Churchill Livingstone,5:0,...,3:0,2:0,1:0,total:0,0,None,282.0,"This book is a detailed, accessible and compre...",NaN,NaN
1470428,3706916,Rheometers for Molten Plastics,John M. Dealy,0442218745,0.00,1982,1,31,Springer,5:0,...,3:0,2:0,1:0,total:0,0,None,302.0,The Society of Plastics Engineers is pleased t...,NaN,NaN
1147621,923381,The Botany of Desire: A Plant's-Eye View of th...,Michael Pollan,1588360083,4.06,2001,12,6,Random House,5:16654,...,3:8553,2:1842,1:581,total:45958,6,eng,NaN,In this original narrative about man and natur...,304.0,6.0
170638,671697,"R.S. Thomas: Conceding an Absence, Images of ...",Elaine Shepherd,0312160984,0.00,1996,15,11,Palgrave Macmillan,5:0,...,3:0,2:0,1:0,total:0,0,None,NaN,R.S. Thomas's presentation of God has given ri...,232.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590984,4346165,Pure Distraction (Pure #1),Carolina Barbour,1606011138,3.65,2008,7,23,Siren Publishing,5:17,...,3:19,2:2,1:5,total:60,4,None,212.0,PURE DISTRACTION Erotic Futuristic Romance]<br...,NaN,NaN
737912,2707425,Serve the Community of the Church: Christians ...,Andrew D. Clarke,0802841821,3.50,2000,1,1,William B. Eerdmans Publishing Company,5:2,...,3:7,2:1,1:0,total:14,1,None,314.0,This volume explores the nature of leadership ...,NaN,NaN
1527688,4148887,The Chemistry of Organic Derivatives of Gold a...,Saul E. Patai,0471981648,0.00,1999,10,18,Wiley,5:0,...,3:0,2:0,1:0,total:0,0,None,764.0,The 100th volume in this highly successful and...,NaN,NaN
1185880,1118164,Greece,Zoran Pavlovic,0791087972,2.50,2006,1,4,Chelsea House Publications,5:0,...,3:1,2:1,1:0,total:2,1,None,NaN,The information-packed volumes in this series ...,104.0,1.0


In [64]:
from tqdm.auto import tqdm
from uuid import uuid4

batch_size = 100

texts = []
metadatas = []

for i in tqdm(range(0, len(data), batch_size)):
    # get end of batch
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    # first get metadata fields for this record
    
    # get the list of contexts / documents
    documents = batch['Description']
    # create document embeddings
    embeds = embed.embed_documents(documents)
    # get IDs
    ids = batch['Id'].astype(str)

  0%|          | 0/1 [00:00<?, ?it/s]

In [69]:
ids.values

array(['655862', '3217053', '3706916', '923381', '671697', '2371657',
       '2187931', '1688608', '1071734', '4555324', '990675', '758461',
       '2496283', '3413372', '1997009', '1301801', '880543', '1886769',
       '696806', '1324657', '2610263', '4804916', '2795234', '1835623',
       '746159', '3703757', '2831890', '2809444', '1429938', '2568464',
       '891416', '684421', '2162366', '3697042', '1572774', '4731647',
       '866830', '2479655', '4409372', '695588', '3315572', '691704',
       '4090954', '4380873', '1174816', '2457468', '1399039', '4574329',
       '2138071', '1463435', '785146', '2737551', '2634244', '1078907',
       '1120724', '2847918', '4414120', '732569', '3183060', '2749140',
       '1169505', '3142764', '3471186', '3563887', '2532858', '1136523',
       '1128769', '893904', '1615291', '4620323', '1282802', '4677377',
       '1820583', '2723022', '4803865', '1981801', '2650570', '2357690',
       '1894844', '3434416', '4020065', '1124442', '792545', '23712

In [67]:
item_feature_matrix_array = np.array(embeds)

# # Print the item feature matrix
print("Item Feature Matrix:\n", item_feature_matrix_array)

Item Feature Matrix:
 [[ 4.84036519e-03 -5.70590520e-03 -7.50356454e-03 ... -1.60990415e-02
   7.64338260e-03 -2.70714241e-02]
 [-2.27802308e-02  1.44087897e-02  1.26614796e-02 ... -2.76179335e-02
   3.08672443e-05 -1.64038529e-02]
 [-4.18550506e-03  1.09261553e-02 -8.09700009e-03 ... -1.55911783e-02
   3.48506661e-03 -4.59994550e-03]
 ...
 [ 3.45274790e-03  2.86118157e-03 -9.71835221e-03 ... -3.09547509e-02
  -2.49157758e-02 -4.85917610e-03]
 [ 1.29910815e-02 -2.98953302e-03  1.75791507e-02 ... -3.71354806e-02
  -6.09313396e-03 -1.49429119e-02]
 [ 9.42202090e-03 -2.13858576e-02 -9.72144949e-03 ... -2.43402196e-02
  -1.49847389e-02 -1.74467078e-02]]


In [70]:
item_feature_matrix = pd.DataFrame(item_feature_matrix_array)
item_feature_matrix.insert(0, 'ID', ids.values)

item_feature_matrix

,ID,0,1,2,3,4,5,6,7,8,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,655862,0.004840,-0.005706,-0.007504,0.002974,-0.009275,0.030334,-0.039628,-0.003768,-0.018056,...,0.019468,-0.027218,0.029295,-0.009761,-0.040081,0.001358,0.004001,-0.016099,0.007643,-0.027071
1,3217053,-0.022780,0.014409,0.012661,-0.046630,0.008300,0.000872,-0.013261,-0.021281,-0.046160,...,0.014748,0.016547,0.024501,-0.009193,-0.000588,-0.033981,-0.014839,-0.027618,0.000031,-0.016404
2,3706916,-0.004186,0.010926,-0.008097,-0.032278,0.003857,0.018071,0.004189,-0.009508,-0.038334,...,-0.000638,-0.001617,0.021798,-0.003737,-0.036334,-0.022250,0.030689,-0.015591,0.003485,-0.004600
3,923381,0.014038,-0.015390,0.009615,-0.020340,-0.020029,0.005305,-0.019893,-0.001257,-0.022192,...,0.008554,-0.022544,0.014403,-0.020461,-0.003298,-0.003601,0.005379,0.002348,-0.008459,-0.006177
4,671697,-0.006686,0.021171,-0.031120,0.025747,0.014313,0.021131,-0.037832,-0.008967,-0.003956,...,-0.007714,-0.025018,0.021821,-0.013272,-0.024620,-0.002713,0.010539,0.000427,0.011580,-0.029767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4346165,-0.005061,-0.022947,-0.023016,-0.021693,-0.003309,0.008443,-0.017959,-0.020021,-0.034580,...,0.010930,-0.008143,0.012825,-0.011794,-0.050157,0.010965,-0.003800,-0.018460,-0.015980,-0.026583
96,2707425,0.018015,-0.003087,-0.029003,-0.038214,-0.009823,0.027318,-0.008139,0.008803,-0.014475,...,0.007586,-0.006672,0.000100,-0.006589,-0.005665,-0.017015,-0.002790,-0.007797,-0.010652,-0.020818
97,4148887,0.003453,0.002861,-0.009718,-0.040766,0.006866,0.031621,-0.023836,-0.032874,-0.050098,...,0.031008,-0.016304,0.046712,-0.011931,-0.023743,-0.019730,0.007725,-0.030955,-0.024916,-0.004859
98,1118164,0.012991,-0.002990,0.017579,-0.019201,-0.018504,0.030596,-0.015830,-0.015247,-0.026971,...,0.019759,0.012193,0.010342,-0.022243,-0.024068,-0.020912,0.006087,-0.037135,-0.006093,-0.014943


In [ ]:
# Get the column names from the 2nd column until the last column
columns_to_rename = item_feature_matrix.columns[1:]

# Create a dictionary of new column names
new_column_names = {column: f'New_{column}' for column in columns_to_rename}

In [196]:
item_feature_matrix

,ID,0,1,2,3,4,5,6,7,8,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,655862,0.004840,-0.005706,-0.007504,0.002974,-0.009275,0.030334,-0.039628,-0.003768,-0.018056,...,0.019468,-0.027218,0.029295,-0.009761,-0.040081,0.001358,0.004001,-0.016099,0.007643,-0.027071
1,3217053,-0.022780,0.014409,0.012661,-0.046630,0.008300,0.000872,-0.013261,-0.021281,-0.046160,...,0.014748,0.016547,0.024501,-0.009193,-0.000588,-0.033981,-0.014839,-0.027618,0.000031,-0.016404
2,3706916,-0.004186,0.010926,-0.008097,-0.032278,0.003857,0.018071,0.004189,-0.009508,-0.038334,...,-0.000638,-0.001617,0.021798,-0.003737,-0.036334,-0.022250,0.030689,-0.015591,0.003485,-0.004600
3,923381,0.014038,-0.015390,0.009615,-0.020340,-0.020029,0.005305,-0.019893,-0.001257,-0.022192,...,0.008554,-0.022544,0.014403,-0.020461,-0.003298,-0.003601,0.005379,0.002348,-0.008459,-0.006177
4,671697,-0.006686,0.021171,-0.031120,0.025747,0.014313,0.021131,-0.037832,-0.008967,-0.003956,...,-0.007714,-0.025018,0.021821,-0.013272,-0.024620,-0.002713,0.010539,0.000427,0.011580,-0.029767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4346165,-0.005061,-0.022947,-0.023016,-0.021693,-0.003309,0.008443,-0.017959,-0.020021,-0.034580,...,0.010930,-0.008143,0.012825,-0.011794,-0.050157,0.010965,-0.003800,-0.018460,-0.015980,-0.026583
96,2707425,0.018015,-0.003087,-0.029003,-0.038214,-0.009823,0.027318,-0.008139,0.008803,-0.014475,...,0.007586,-0.006672,0.000100,-0.006589,-0.005665,-0.017015,-0.002790,-0.007797,-0.010652,-0.020818
97,4148887,0.003453,0.002861,-0.009718,-0.040766,0.006866,0.031621,-0.023836,-0.032874,-0.050098,...,0.031008,-0.016304,0.046712,-0.011931,-0.023743,-0.019730,0.007725,-0.030955,-0.024916,-0.004859
98,1118164,0.012991,-0.002990,0.017579,-0.019201,-0.018504,0.030596,-0.015830,-0.015247,-0.026971,...,0.019759,0.012193,0.010342,-0.022243,-0.024068,-0.020912,0.006087,-0.037135,-0.006093,-0.014943


In [205]:
item_feature_matrix.columns = [str(x) for x in range(-1, 1536)]

In [206]:
item_feature_matrix

,-1,0,1,2,3,4,5,6,7,8,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,655862,0.004840,-0.005706,-0.007504,0.002974,-0.009275,0.030334,-0.039628,-0.003768,-0.018056,...,0.019468,-0.027218,0.029295,-0.009761,-0.040081,0.001358,0.004001,-0.016099,0.007643,-0.027071
1,3217053,-0.022780,0.014409,0.012661,-0.046630,0.008300,0.000872,-0.013261,-0.021281,-0.046160,...,0.014748,0.016547,0.024501,-0.009193,-0.000588,-0.033981,-0.014839,-0.027618,0.000031,-0.016404
2,3706916,-0.004186,0.010926,-0.008097,-0.032278,0.003857,0.018071,0.004189,-0.009508,-0.038334,...,-0.000638,-0.001617,0.021798,-0.003737,-0.036334,-0.022250,0.030689,-0.015591,0.003485,-0.004600
3,923381,0.014038,-0.015390,0.009615,-0.020340,-0.020029,0.005305,-0.019893,-0.001257,-0.022192,...,0.008554,-0.022544,0.014403,-0.020461,-0.003298,-0.003601,0.005379,0.002348,-0.008459,-0.006177
4,671697,-0.006686,0.021171,-0.031120,0.025747,0.014313,0.021131,-0.037832,-0.008967,-0.003956,...,-0.007714,-0.025018,0.021821,-0.013272,-0.024620,-0.002713,0.010539,0.000427,0.011580,-0.029767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4346165,-0.005061,-0.022947,-0.023016,-0.021693,-0.003309,0.008443,-0.017959,-0.020021,-0.034580,...,0.010930,-0.008143,0.012825,-0.011794,-0.050157,0.010965,-0.003800,-0.018460,-0.015980,-0.026583
96,2707425,0.018015,-0.003087,-0.029003,-0.038214,-0.009823,0.027318,-0.008139,0.008803,-0.014475,...,0.007586,-0.006672,0.000100,-0.006589,-0.005665,-0.017015,-0.002790,-0.007797,-0.010652,-0.020818
97,4148887,0.003453,0.002861,-0.009718,-0.040766,0.006866,0.031621,-0.023836,-0.032874,-0.050098,...,0.031008,-0.016304,0.046712,-0.011931,-0.023743,-0.019730,0.007725,-0.030955,-0.024916,-0.004859
98,1118164,0.012991,-0.002990,0.017579,-0.019201,-0.018504,0.030596,-0.015830,-0.015247,-0.026971,...,0.019759,0.012193,0.010342,-0.022243,-0.024068,-0.020912,0.006087,-0.037135,-0.006093,-0.014943


In [207]:
item_feature_matrix.to_parquet('../data/item_feature_matrix_sample.parquet')

#### Generate Embeddings Spark

In [119]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.functions import PandasUDFType
from pyspark.sql.types import StructType, StructField, LongType, ArrayType, FloatType
import pandas as pd
import openai
import numpy as np

In [120]:
spark = SparkSession.builder.appName("OpenAIEmbeddings").getOrCreate()

In [121]:
schema = StructType([
    StructField("book_id", LongType(), False),
    StructField("vector_embeddings", ArrayType(FloatType()), False)
])

In [173]:
@pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)
def generate_embeddings(doc):
    # List to hold embeddings
    embeddings_list = []

    # Batch request to OpenAI embed API
    # Please note that OpenAI has a max batch size, so consider that here.
    responses = openai.embeddings.create(
        model="text-embedding-ada-002",  
        input=doc['Description'].tolist()
    )

    # Retrieve the embeddings from OpenAI responses
    for response in responses.data[0]:
        embeddings_list.append(response.embedding)

    # Return the new DataFrame
    return pd.DataFrame({
        # 'book_id': doc['book_id'],
        # 'vector_embeddings': embeddings_list
        'test': input
    })


In [167]:
df = spark.read.csv("../data/sample_books_cleaned.csv", header=True, inferSchema=True)
# sample_books_cleaned.to_csv('../data/sample_books_cleaned.csv')


In [168]:
repartitioned_df = df.repartition(10)

In [99]:
spark.stop()

#### Embeddings FlagModel

In [172]:
import dask.dataframe as dd
from dask.multiprocessing import get

In [174]:
df = sample_book_cleaned.copy()

In [26]:
ddata = dd.from_pandas(df[['Id', 'Description']], npartitions=30)
ddata

,Id,Description
npartitions=30,,
632,int64,object
2685,...,...
...,...,...
115572,...,...
121574,...,...


In [27]:
from FlagEmbedding import FlagModel

model = FlagModel('BAAI/bge-large-en-v1.5', 
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages: ",
                  use_fp16=False)


In [28]:
def embed_text(doc):
    return model.encode(doc)

In [29]:
ddata['DescEmbeddings'] = ddata['Description'].apply(embed_text, meta = ('Description', 'object'))

In [30]:
embeddings_df = ddata.compute()

In [66]:
embeddings_df

,Id,Description,DescEmbeddings
632,1915342,"Never before has there been a comprehensive, i...","[0.019671917, -0.010679969, 0.026023379, -0.01..."
1749,1945041,"This trilogy tells the story of Orlando King, ...","[0.027494812, -0.029625617, 0.0065872455, 0.00..."
2327,1962492,This anthology contains: Introduction by Rober...,"[0.016765222, -0.034488402, 0.016900223, 0.038..."
2448,1966495,History and fiction collide with deadly conseq...,"[0.038851917, -0.046406824, 0.0014487669, 0.01..."
2685,1974029,Australia has been invaded. Nothing is as it w...,"[0.034037847, -0.0056402422, -0.021395188, 0.0..."
...,...,...,...
113806,4458655,"This anthology contains a wide range of poems,...","[-0.038541548, -0.009716585, 0.06514758, -0.00..."
114648,4538410,Explore the daily lives of ancient Egyptians i...,"[-0.007341567, 0.016864993, 0.009468778, -0.00..."
115572,4619663,"This unforgettable tale, illustrated by Caldec...","[0.016770082, 0.007270308, 0.0013751144, 0.019..."
117988,4838239,"Out of Print<br /><br /><i>""A book in The Huss...","[0.033698164, 0.015909215, -0.022079729, 0.050..."


In [17]:
embeddings = pd.DataFrame(embeddings_df['DescEmbeddings'].tolist())
embeddings.columns = [str(x) for x in range(embeddings.shape[1])]
embeddings.insert(0, 'Id', embeddings_df['Id'].tolist())
embeddings

,Id,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1915342,0.019672,-0.010680,0.026023,-0.012980,-0.011049,-0.042468,-0.027986,-0.046368,0.013329,...,0.039828,0.017217,-0.034920,0.008253,0.021344,0.012759,-0.014850,-0.001685,0.029440,0.026171
1,1945041,0.027495,-0.029626,0.006587,0.007185,-0.002153,-0.036707,-0.075638,-0.008426,0.027270,...,0.030011,0.028950,-0.026864,-0.002844,0.017709,0.022555,-0.029902,-0.012736,-0.017359,0.018014
2,1962492,0.016765,-0.034488,0.016900,0.038029,0.000625,-0.050714,-0.033872,-0.014772,0.014385,...,0.035867,0.005906,0.002038,-0.013409,-0.002867,0.022737,-0.001784,-0.009821,0.041732,-0.028875
3,1966495,0.038852,-0.046407,0.001449,0.019194,0.012761,-0.079849,-0.044601,-0.027145,-0.005161,...,0.058807,-0.001943,-0.023606,-0.014104,0.037743,0.029775,-0.005517,0.018269,0.012411,-0.004323
4,1974029,0.034038,-0.005640,-0.021395,0.031956,0.015900,-0.051513,-0.046776,0.019667,0.008174,...,0.016996,-0.014675,-0.025072,0.005797,-0.014186,0.036789,0.014259,-0.022536,-0.013623,0.002711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4458655,-0.038542,-0.009717,0.065148,-0.008847,-0.005102,-0.016512,0.027890,0.031256,0.025674,...,0.042025,-0.032761,-0.027181,-0.039687,-0.005633,0.033426,0.018797,-0.007476,0.016977,-0.040620
96,4538410,-0.007342,0.016865,0.009469,-0.008030,-0.013015,-0.001679,-0.056379,0.017555,0.044251,...,0.055220,-0.012609,-0.042055,0.003586,0.043074,0.036913,0.009439,-0.027818,-0.020053,-0.007549
97,4619663,0.016770,0.007270,0.001375,0.019624,0.012343,-0.048061,-0.019651,0.016987,0.033504,...,0.053097,-0.016903,0.026127,0.003713,0.005002,0.037359,0.023257,0.008849,0.017564,0.004834
98,4838239,0.033698,0.015909,-0.022080,0.050326,0.008085,-0.007000,-0.007634,0.031579,0.029642,...,0.049828,-0.026999,-0.011276,0.016474,0.026631,0.024221,0.026163,0.017199,0.049599,-0.024829


In [18]:
embeddings.to_parquet('../data/item_feature_matrix_sample_dask.parquet')

### Generate Title Embeddings

In [175]:
ddata = dd.from_pandas(df[['Id', 'Name']], npartitions=30)
ddata

,Id,Name
npartitions=30,,
632,int64,object
2685,...,...
...,...,...
115572,...,...
121574,...,...


In [136]:
embeddings = df['Description'].apply(embed_text)
embeddings

54015     [0.016871113, -0.014289217, 0.0060686423, 0.00...
96931     [0.011660287, 0.015110543, -0.031430896, -0.02...
105559    [0.044182066, -0.01539918, -0.0057495343, 0.05...
75202     [0.019515036, -0.03227013, -0.00838484, 0.0336...
89816     [0.02507292, -0.027432699, -0.016972598, 0.018...
                                ...                        
121574    [-0.03929219, 0.023619993, -0.014403046, -0.00...
82239     [0.0017016538, -0.0034101964, -0.03638816, 0.0...
88653     [0.018419705, -0.033249445, -0.007858579, 0.01...
20356     [0.04216978, 0.004726363, 0.012943139, -0.0261...
34548     [0.008475162, 0.0127646895, -0.0416514, 0.0135...
Name: Description, Length: 100, dtype: object

In [140]:
embeddings_df = pd.DataFrame(embeddings.tolist())
embeddings_df.columns = [str(x) for x in range(embeddings_df.shape[1])]
embeddings_df.insert(0, 'Id', df['Id'].tolist())
embeddings_df

,Id,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,2167181,0.016871,-0.014289,0.006069,0.006901,-0.000575,-0.028419,-0.031998,0.006701,-0.006108,...,0.044029,-0.006014,-0.015839,0.001971,0.007722,0.019793,-0.040538,0.027163,0.012334,0.028178
1,3069617,0.011660,0.015111,-0.031431,-0.029204,-0.023611,-0.040591,-0.004589,0.048419,0.017929,...,0.013596,-0.044199,-0.024734,-0.050729,0.038620,0.031805,0.026043,-0.002245,-0.015313,0.005503
2,3430781,0.044182,-0.015399,-0.005750,0.054954,0.000253,-0.021884,0.004243,0.023303,-0.072284,...,0.038711,-0.027899,0.026159,0.014439,0.062457,0.030255,-0.004003,0.022742,-0.028293,-0.064190
3,2971345,0.019515,-0.032270,-0.008385,0.033656,-0.025756,-0.023265,-0.046986,0.032251,0.018391,...,0.026283,0.009768,-0.036358,-0.040202,0.054221,0.023555,0.027178,-0.010965,0.004345,0.005805
4,1108028,0.025073,-0.027433,-0.016973,0.018087,-0.019807,-0.019906,0.004375,0.021954,0.034171,...,0.021489,0.024228,-0.027435,-0.031106,0.009198,0.007005,0.010375,-0.012862,0.011688,0.014282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1484139,-0.039292,0.023620,-0.014403,-0.006401,-0.004512,-0.038431,0.038959,-0.014227,-0.002381,...,0.025358,-0.020934,0.049738,-0.020004,0.036521,-0.001323,0.059343,0.016928,0.026969,0.025706
96,1886108,0.001702,-0.003410,-0.036388,0.054165,-0.033801,-0.044523,0.005675,0.019056,-0.013114,...,0.041248,-0.039678,-0.002356,-0.013519,0.000028,-0.014913,-0.000805,-0.007656,0.033420,-0.034740
97,989125,0.018420,-0.033249,-0.007859,0.010590,-0.012348,0.018414,-0.007415,0.011025,0.026387,...,0.044350,0.005563,-0.028686,-0.068962,0.016592,0.031268,0.002998,0.002450,-0.036882,0.036958
98,686352,0.042170,0.004726,0.012943,-0.026190,0.006399,-0.037676,-0.004517,0.016114,-0.011290,...,0.024825,-0.033248,0.018162,-0.048114,0.004966,0.032374,-0.065015,0.022778,-0.002493,-0.014655


In [20]:
ddata['TitleEmbeddings'] = ddata['Name'].apply(embed_text, meta = ('Name', 'object'))

In [21]:
embeddings_df = ddata.compute()

In [22]:
embeddings = pd.DataFrame(embeddings_df['TitleEmbeddings'].tolist())
embeddings.columns = [str(x) for x in range(embeddings.shape[1])]
embeddings.insert(0, 'Id', embeddings_df['Id'].tolist())
embeddings

,Id,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1915342,0.031399,-0.024493,0.012370,-0.005739,-0.020743,-0.059919,-0.036419,-0.042084,0.004719,...,0.024711,-0.001506,-0.006059,0.007519,0.027024,0.010831,-0.031846,0.011259,0.021093,0.027407
1,1945041,0.021745,-0.016952,0.020417,-0.011403,-0.010153,-0.018079,-0.024769,-0.007973,0.029864,...,0.030106,0.032769,-0.001989,-0.008776,0.013326,0.001546,-0.034584,-0.018282,-0.000191,-0.004799
2,1962492,0.025225,-0.000295,0.012156,0.021322,0.011494,-0.034446,-0.041135,-0.026693,-0.007239,...,0.002216,0.001883,0.005547,-0.026220,0.022226,0.010203,0.041052,-0.012110,0.033955,-0.021945
3,1966495,0.007741,-0.015637,0.016903,0.004039,-0.011639,-0.096463,-0.013699,-0.000347,-0.014831,...,-0.001014,-0.012654,-0.033562,-0.036246,0.027374,0.019149,-0.010276,0.008718,-0.002202,0.000216
4,1974029,0.022673,0.011059,-0.042224,0.008642,-0.015433,-0.048572,-0.060135,0.021256,-0.009187,...,0.008759,-0.006051,0.009320,-0.003143,-0.022458,0.049964,0.016950,-0.028461,-0.040786,-0.018901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4458655,-0.031841,-0.023764,0.045918,-0.046154,0.019710,-0.036336,0.028953,0.024803,0.021356,...,0.036628,-0.007497,0.018323,-0.039291,0.035235,0.039669,-0.013801,-0.007832,0.004409,0.004863
96,4538410,0.010488,0.020760,0.010961,-0.020280,-0.020682,0.006781,-0.055386,-0.002088,0.041425,...,0.065677,-0.009558,0.012826,0.000350,0.037277,0.017887,-0.009633,-0.018048,-0.024166,0.011467
97,4619663,0.010956,-0.012829,0.034672,0.012926,0.021252,-0.019998,-0.011179,0.040223,0.054586,...,0.049847,0.002667,0.014616,-0.001504,0.043358,0.017657,0.044806,-0.018140,0.025231,-0.001165
98,4838239,0.020996,0.021444,-0.001163,0.008398,-0.021590,-0.029452,0.008113,0.044566,0.019881,...,0.013262,-0.034436,0.016029,0.008735,0.014723,0.070592,0.010350,-0.019286,0.000652,-0.007991


In [23]:
embeddings.to_parquet('../data/item_feature_matrix_sample_dask_title.parquet')

### Item Similarity

In [1]:
from math import pow, sqrt
from decimal import Decimal
import multiprocessing as mp
import pandas as pd
import numpy as np
from itertools import repeat
from ComputeSimilarity import SimilarityMetric


#### Description

In [153]:
item_feature_matrix_desc = pd.read_parquet('../data/item_feature_matrix_sample_dask.parquet')

In [155]:
embeddings_1 = item_feature_matrix_desc.iloc[:, 1:]
embeddings_2 = item_feature_matrix_desc.iloc[:, 1:]

In [156]:
similarity_desc = embeddings_1 @ embeddings_2.T
similarity_desc

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.526837,0.632033,0.695036,0.609767,0.432784,0.603110,0.553023,0.586080,0.629477,...,0.522748,0.443133,0.566123,0.513377,0.491066,0.565220,0.555310,0.440485,0.587951,0.451446
1,0.526837,1.000000,0.492371,0.584183,0.546633,0.520349,0.472885,0.479328,0.565793,0.536691,...,0.389789,0.382276,0.400009,0.468904,0.371931,0.443956,0.454804,0.472685,0.444639,0.357569
2,0.632033,0.492371,1.000000,0.562633,0.582954,0.423598,0.555453,0.503359,0.591040,0.566976,...,0.491421,0.434629,0.477382,0.452820,0.414231,0.564213,0.533175,0.445572,0.574719,0.380325
3,0.695036,0.584183,0.562633,1.000000,0.662237,0.473374,0.594470,0.564176,0.618939,0.591547,...,0.476985,0.472240,0.495777,0.481265,0.414492,0.448497,0.557470,0.503485,0.609517,0.397961
4,0.609767,0.546633,0.582954,0.662237,1.000000,0.541585,0.602263,0.572613,0.651075,0.570456,...,0.463763,0.503836,0.522584,0.566721,0.449152,0.512616,0.552612,0.492091,0.629949,0.385610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.565220,0.443956,0.564213,0.448497,0.512616,0.393383,0.577753,0.576663,0.511504,0.587230,...,0.458717,0.459695,0.474869,0.503495,0.454621,1.000000,0.496633,0.470337,0.484930,0.430948
96,0.555310,0.454804,0.533175,0.557470,0.552612,0.455829,0.524547,0.523625,0.557234,0.524119,...,0.601306,0.465643,0.579992,0.547912,0.578586,0.496633,1.000000,0.394734,0.549726,0.497714
97,0.440485,0.472685,0.445572,0.503485,0.492091,0.400360,0.510944,0.496533,0.501004,0.476172,...,0.328708,0.510738,0.421187,0.420239,0.294487,0.470337,0.394734,1.000000,0.444224,0.429589
98,0.587951,0.444639,0.574719,0.609517,0.629949,0.481018,0.560959,0.545550,0.671933,0.552091,...,0.443255,0.387622,0.500390,0.442378,0.420970,0.484930,0.549726,0.444224,1.000000,0.404849


#### Title

In [157]:
item_feature_matrix_title = pd.read_parquet('../data/item_feature_matrix_sample_dask_title.parquet')

In [158]:
item_feature_matrix_title

,Id,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1915342,0.031399,-0.024493,0.012370,-0.005739,-0.020743,-0.059919,-0.036419,-0.042084,0.004719,...,0.024711,-0.001506,-0.006059,0.007519,0.027024,0.010831,-0.031846,0.011259,0.021093,0.027407
1,1945041,0.021745,-0.016952,0.020417,-0.011403,-0.010153,-0.018079,-0.024769,-0.007973,0.029864,...,0.030106,0.032769,-0.001989,-0.008776,0.013326,0.001546,-0.034584,-0.018282,-0.000191,-0.004799
2,1962492,0.025225,-0.000295,0.012156,0.021322,0.011494,-0.034446,-0.041135,-0.026693,-0.007239,...,0.002216,0.001883,0.005547,-0.026220,0.022226,0.010203,0.041052,-0.012110,0.033955,-0.021945
3,1966495,0.007741,-0.015637,0.016903,0.004039,-0.011639,-0.096463,-0.013699,-0.000347,-0.014831,...,-0.001014,-0.012654,-0.033562,-0.036246,0.027374,0.019149,-0.010276,0.008718,-0.002202,0.000216
4,1974029,0.022673,0.011059,-0.042224,0.008642,-0.015433,-0.048572,-0.060135,0.021256,-0.009187,...,0.008759,-0.006051,0.009320,-0.003143,-0.022458,0.049964,0.016950,-0.028461,-0.040786,-0.018901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4458655,-0.031841,-0.023764,0.045918,-0.046154,0.019710,-0.036336,0.028953,0.024803,0.021356,...,0.036628,-0.007497,0.018323,-0.039291,0.035235,0.039669,-0.013801,-0.007832,0.004409,0.004863
96,4538410,0.010488,0.020760,0.010961,-0.020280,-0.020682,0.006781,-0.055386,-0.002088,0.041425,...,0.065677,-0.009558,0.012826,0.000350,0.037277,0.017887,-0.009633,-0.018048,-0.024166,0.011467
97,4619663,0.010956,-0.012829,0.034672,0.012926,0.021252,-0.019998,-0.011179,0.040223,0.054586,...,0.049847,0.002667,0.014616,-0.001504,0.043358,0.017657,0.044806,-0.018140,0.025231,-0.001165
98,4838239,0.020996,0.021444,-0.001163,0.008398,-0.021590,-0.029452,0.008113,0.044566,0.019881,...,0.013262,-0.034436,0.016029,0.008735,0.014723,0.070592,0.010350,-0.019286,0.000652,-0.007991


In [159]:
embeddings_1 = item_feature_matrix_title.iloc[:, 1:]
embeddings_2 = item_feature_matrix_title.iloc[:, 1:]

In [162]:
similarity_title = embeddings_1 @ embeddings_2.T
similarity_title

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.603574,0.632852,0.625820,0.571715,0.540332,0.617729,0.547925,0.434457,0.489652,...,0.490650,0.585662,0.407596,0.489122,0.385286,0.560283,0.517047,0.553936,0.442693,0.603432
1,0.603574,1.000000,0.583939,0.539316,0.536227,0.549630,0.568717,0.549123,0.523106,0.443251,...,0.469721,0.524558,0.447282,0.453681,0.453342,0.534956,0.488406,0.528274,0.415995,0.544936
2,0.632852,0.583939,1.000000,0.549668,0.640464,0.525405,0.593066,0.608516,0.517556,0.439011,...,0.556982,0.696019,0.428357,0.514377,0.443147,0.525794,0.545869,0.561426,0.486108,0.578829
3,0.625820,0.539316,0.549668,1.000000,0.523906,0.553210,0.533509,0.547865,0.526312,0.508759,...,0.499189,0.627802,0.456116,0.493425,0.413887,0.489728,0.487009,0.511204,0.480534,0.558408
4,0.571715,0.536227,0.640464,0.523906,1.000000,0.614153,0.468621,0.503807,0.484232,0.483189,...,0.427202,0.511246,0.450501,0.426019,0.401860,0.480121,0.487839,0.486193,0.437767,0.479306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.560283,0.534956,0.525794,0.489728,0.480121,0.408745,0.591845,0.583033,0.475704,0.499320,...,0.491274,0.581279,0.427657,0.530306,0.387215,1.000000,0.509032,0.695385,0.437482,0.565131
96,0.517047,0.488406,0.545869,0.487009,0.487839,0.460518,0.486505,0.515376,0.446155,0.436571,...,0.484254,0.566870,0.457025,0.491909,0.421372,0.509032,1.000000,0.544015,0.396010,0.598004
97,0.553936,0.528274,0.561426,0.511204,0.486193,0.513255,0.607760,0.598682,0.511940,0.455124,...,0.470722,0.570664,0.383904,0.503044,0.436734,0.695385,0.544015,1.000000,0.430654,0.593926
98,0.442693,0.415995,0.486108,0.480534,0.437767,0.463486,0.542893,0.402455,0.444003,0.463148,...,0.490636,0.456226,0.451800,0.375589,0.437330,0.437482,0.396010,0.430654,1.000000,0.421809


##### Non-parallel

In [160]:
embeddings = embeddings_1
similarity_matrix = np.matrix(np.zeros((embeddings.shape[0], embeddings.shape[0],)))
for i in range(0,embeddings.shape[0]):
    for j in range(0,embeddings.shape[0]):
        similarity_matrix[i, j] = np.sum(embeddings.iloc[i, :] * embeddings.iloc[j, :])

In [161]:
final = pd.DataFrame(similarity_matrix)
final.columns = item_feature_matrix_title['Id'].tolist()
final.index = item_feature_matrix_title['Id'].tolist()
final.index = final_similarity_matrix.columns.astype('str')
final.index = final_similarity_matrix.index.astype('str')
final


,1915342,1945041,1962492,1966495,1974029,1629841,1630607,1639088,1646536,1690826,...,4055855,4191602,4293954,4375030,4413662,4458655,4538410,4619663,4838239,1484139
1915342,1.000000,0.603574,0.632852,0.625820,0.571715,0.540333,0.617729,0.547925,0.434457,0.489652,...,0.490651,0.585662,0.407597,0.489122,0.385286,0.560283,0.517047,0.553936,0.442693,0.603432
1945041,0.603574,1.000000,0.583939,0.539316,0.536227,0.549630,0.568717,0.549123,0.523106,0.443251,...,0.469721,0.524558,0.447282,0.453681,0.453342,0.534956,0.488406,0.528274,0.415995,0.544936
1962492,0.632852,0.583939,1.000000,0.549668,0.640464,0.525405,0.593066,0.608516,0.517556,0.439011,...,0.556982,0.696019,0.428357,0.514377,0.443147,0.525794,0.545869,0.561426,0.486108,0.578829
1966495,0.625820,0.539316,0.549668,1.000000,0.523906,0.553210,0.533509,0.547865,0.526313,0.508759,...,0.499189,0.627801,0.456116,0.493425,0.413887,0.489728,0.487009,0.511204,0.480534,0.558408
1974029,0.571715,0.536227,0.640464,0.523906,1.000000,0.614153,0.468621,0.503807,0.484232,0.483189,...,0.427202,0.511246,0.450501,0.426019,0.401860,0.480121,0.487839,0.486193,0.437767,0.479306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4458655,0.560283,0.534956,0.525794,0.489728,0.480121,0.408745,0.591845,0.583033,0.475704,0.499320,...,0.491274,0.581279,0.427657,0.530306,0.387215,1.000000,0.509032,0.695385,0.437481,0.565131
4538410,0.517047,0.488406,0.545869,0.487009,0.487839,0.460518,0.486505,0.515376,0.446155,0.436571,...,0.484254,0.566870,0.457025,0.491909,0.421372,0.509032,1.000000,0.544015,0.396010,0.598004
4619663,0.553936,0.528274,0.561426,0.511204,0.486193,0.513255,0.607760,0.598682,0.511940,0.455124,...,0.470722,0.570664,0.383904,0.503044,0.436734,0.695385,0.544015,1.000000,0.430654,0.593926
4838239,0.442693,0.415995,0.486108,0.480534,0.437767,0.463486,0.542893,0.402455,0.444003,0.463148,...,0.490636,0.456226,0.451800,0.375589,0.437330,0.437481,0.396010,0.430654,1.000000,0.421809


#### Final Similarity Matrix

In [163]:
final_similarity_matrix = 0.4*similarity_title + 0.6*similarity_desc
final_similarity_matrix

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.557532,0.632361,0.667350,0.594546,0.475803,0.608958,0.550984,0.525431,0.573547,...,0.509909,0.500144,0.502713,0.503675,0.448754,0.563245,0.540005,0.485866,0.529848,0.512240
1,0.557532,1.000000,0.528998,0.566236,0.542471,0.532062,0.511218,0.507246,0.548718,0.499315,...,0.421762,0.439189,0.418918,0.462815,0.404495,0.480356,0.468245,0.494921,0.433182,0.432516
2,0.632361,0.528998,1.000000,0.557447,0.605958,0.464321,0.570498,0.545422,0.561647,0.515790,...,0.517645,0.539185,0.457772,0.477443,0.425798,0.548845,0.538253,0.491913,0.539275,0.459726
3,0.667350,0.566236,0.557447,1.000000,0.606905,0.505309,0.570086,0.557652,0.581888,0.558432,...,0.485866,0.534465,0.479913,0.486129,0.414250,0.464989,0.529285,0.506573,0.557923,0.462140
4,0.594546,0.542471,0.605958,0.606905,1.000000,0.570612,0.548806,0.545091,0.584338,0.535549,...,0.449139,0.506800,0.493751,0.510440,0.430236,0.499618,0.526703,0.489732,0.553077,0.423088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.563245,0.480356,0.548845,0.464989,0.499618,0.399528,0.583390,0.579211,0.497184,0.552066,...,0.471740,0.508328,0.455984,0.514220,0.427658,1.000000,0.501593,0.560356,0.465950,0.484621
96,0.540005,0.468245,0.538253,0.529285,0.526703,0.457705,0.509330,0.520326,0.512803,0.489100,...,0.554485,0.506134,0.530805,0.525511,0.515701,0.501593,1.000000,0.454446,0.488240,0.537830
97,0.485866,0.494921,0.491913,0.506573,0.489732,0.445518,0.549671,0.537393,0.505378,0.467753,...,0.385514,0.534708,0.406274,0.453361,0.351386,0.560356,0.454446,1.000000,0.438796,0.495324
98,0.529848,0.433182,0.539275,0.557923,0.553077,0.474006,0.553733,0.488312,0.580761,0.516514,...,0.462208,0.415064,0.480954,0.415662,0.427514,0.465950,0.488240,0.438796,1.000000,0.411633


In [169]:
final_similarity_matrix.columns = item_feature_matrix_title['Id'].tolist()
final_similarity_matrix.index = item_feature_matrix_title['Id'].tolist()
final_similarity_matrix.columns = final_similarity_matrix.columns.astype('str')
final_similarity_matrix.index = final_similarity_matrix.index.astype('str')

In [170]:
final_similarity_matrix.to_parquet('../data/final_item_similarity_matrix_sample.parquet')

### Create User Item Matrix (Ratings)

In [146]:
existing_sample_books = final_similarity_matrix.columns.tolist()
existing_sample_books = [str(x) for x in existing_sample_books]

In [147]:
user_rating_df = pd.read_parquet('../data/user_rating_total.parquet')
user_rating_df

,ID,Name,book_id,Rating
0,1,Agile Web Development with Rails: A Pragmatic ...,45.0,5
1,1,The Restaurant at the End of the Universe (Hit...,862825.0,5
2,1,The Restaurant at the End of the Universe (Hit...,8695.0,5
3,1,Siddhartha,1943716.0,5
4,1,Siddhartha,1646972.0,5
...,...,...,...,...
1187663,5403,"Dance Upon The Air (Three Sisters Island, #1)",1479796.0,4
1187664,5403,"Valley Of Silence (Circle Trilogy, #3)",636713.0,3
1187665,5403,"Valley Of Silence (Circle Trilogy, #3)",114123.0,3
1187666,5403,"Valley Of Silence (Circle Trilogy, #3)",1705621.0,3


In [148]:
user_rating_df_cleaned = user_rating_df.drop_duplicates()
user_rating_df_cleaned.dropna(inplace=True)
user_rating_df_cleaned.reset_index(drop = True, inplace = True)


/var/folders/cg/24np02xs7q5f9700x29kzs180000gn/T/ipykernel_53173/2893867180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_rating_df_cleaned.dropna(inplace=True)


In [149]:
user_rating_df_cleaned.book_id = user_rating_df_cleaned.book_id.astype('int').astype('str')
user_rating_df_cleaned

/var/folders/cg/24np02xs7q5f9700x29kzs180000gn/T/ipykernel_53173/3738182023.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_rating_df_cleaned.book_id = user_rating_df_cleaned.book_id.astype('int').astype('str')


,ID,Name,book_id,Rating
0,1,Agile Web Development with Rails: A Pragmatic ...,45,5
1,1,The Restaurant at the End of the Universe (Hit...,862825,5
2,1,The Restaurant at the End of the Universe (Hit...,8695,5
3,1,Siddhartha,1943716,5
4,1,Siddhartha,1646972,5
...,...,...,...,...
1055634,5403,"Dance Upon The Air (Three Sisters Island, #1)",1479796,4
1055635,5403,"Valley Of Silence (Circle Trilogy, #3)",636713,3
1055636,5403,"Valley Of Silence (Circle Trilogy, #3)",114123,3
1055637,5403,"Valley Of Silence (Circle Trilogy, #3)",1705621,3


In [71]:
sample_user_rating = user_rating_df_cleaned[user_rating_df_cleaned['book_id'].isin(existing_sample_books)]

In [72]:
sample_user_rating

,ID,Name,book_id,Rating
23382,166,"The Spellman Files (The Spellmans, #1)",2167181,3
32886,192,The Best of Fritz Leiber,606215,4
34664,201,"The Well of Ascension (Mistborn, #2)",2115046,4
49110,284,"Incubus Dreams (Anita Blake, Vampire Hunter, #12)",1080513,1
85537,464,"Incubus Dreams (Anita Blake, Vampire Hunter, #12)",1080513,3
...,...,...,...,...
1019460,5111,"The Unexpected Mrs. Pollifax (Mrs. Pollifax, #1)",1375925,4
1037502,4534,"Incubus Dreams (Anita Blake, Vampire Hunter, #12)",1080513,4
1044647,4489,The Vagrants,920766,4
1045127,4534,Talking to Dragons (Enchanted Forest Chronicle...,2833823,3


In [73]:
sample_user_rating.book_id = sample_user_rating.book_id.astype('int').astype('str')
sample_user_rating.ID = sample_user_rating.ID.astype('str')

/var/folders/cg/24np02xs7q5f9700x29kzs180000gn/T/ipykernel_53173/2532159943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_user_rating.book_id = sample_user_rating.book_id.astype('int').astype('str')
/var/folders/cg/24np02xs7q5f9700x29kzs180000gn/T/ipykernel_53173/2532159943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_user_rating.ID = sample_user_rating.ID.astype('str')


In [19]:
sample_user_rating.groupby(['ID', 'book_id']).size().reset_index().rename(columns={0:'count'}).sort_values('count')

,ID,book_id,count
0,1045,2115046,1
64,7417,2167181,1
63,7410,1375925,1
62,7396,2115046,1
61,7328,1012959,1
...,...,...,...
26,3286,1802466,1
25,3286,1375925,1
24,3259,1080513,1
22,3017,1646536,1


In [107]:
ddata = dd.from_pandas(sample_user_rating[['ID', 'book_id', 'Rating']], npartitions=30)
ddata_aggregated = ddata.groupby(['ID', 'book_id']).aggregate('mean')

In [116]:
ddata_aggregated

,Rating
npartitions=1,
,float64
,...


In [121]:
df_aggregated = ddata_aggregated.compute().reset_index()
user_item_matrix = df_aggregated.pivot(index='ID', columns='book_id', values='Rating').fillna(0)

In [74]:
df_aggregated = sample_user_rating.groupby(['ID', 'book_id']).mean().reset_index()

/var/folders/cg/24np02xs7q5f9700x29kzs180000gn/T/ipykernel_53173/3646733090.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_aggregated = sample_user_rating.groupby(['ID', 'book_id']).mean().reset_index()


In [75]:
user_item_matrix = df_aggregated.pivot(index='ID', columns='book_id', values='Rating').fillna(0)
user_item_matrix

book_id,1012959,1080513,1375925,1646536,1690826,1802466,1843325,2115046,2167181,2238834,2833823,2993119,3172076,606215,776434,849880,920766,997117
ID,,,,,,,,,,,,,,,,,,
1045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1071,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
10857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
user_item_matrix.to_parquet('../data/user_item_matrix_sample.parquet')